<a href="https://colab.research.google.com/github/Giulero/ADAM/blob/main/examples/mpc-ik.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
!pip install mujoco
!pip install adam-robotics
!git clone https://github.com/google-deepmind/mujoco_menagerie.git
!pip install -q mediapy
!wget https://raw.githubusercontent.com/bulletphysics/bullet3/master/examples/pybullet/gym/pybullet_data/franka_panda/panda.urdf
import distutils.util
import os
import subprocess
# Graphics and plotting.
print('Installing mediapy:')
!command -v ffmpeg >/dev/null || (apt update && apt install -y ffmpeg)

# Configure MuJoCo to use the EGL rendering backend (requires GPU)
print('Setting environment variable to use GPU rendering:')
%env MUJOCO_GL=egl


fatal: destination path 'mujoco_menagerie' already exists and is not an empty directory.
--2023-12-07 14:27:01--  https://raw.githubusercontent.com/bulletphysics/bullet3/master/examples/pybullet/gym/pybullet_data/franka_panda/panda.urdf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11263 (11K) [text/plain]
Saving to: ‘panda.urdf.1’

panda.urdf.1        100%[===================>]  11.00K  --.-KB/s    in 0s      

2023-12-07 14:27:01 (51.5 MB/s) - ‘panda.urdf.1’ saved [11263/11263]

Installing mediapy:
Setting environment variable to use GPU rendering:
env: MUJOCO_GL=egl
env: OMP_NUM_THREADS=1


# Nuova sezione

In [17]:
os.environ["OMP_NUM_THREADS"] = "1"
import mujoco
import mediapy as media
from adam.casadi import KinDynComputations
import numpy as np
import casadi as cs


In [3]:
# load scene from xml
model = mujoco.MjModel.from_xml_path("mujoco_menagerie/franka_emika_panda/scene.xml")


In [13]:
# load the urdf in adam
# the importer directly from a mujoco model is not yet implemented

joints_name_list = ['panda_joint1', 'panda_joint2', 'panda_joint3', 'panda_joint4', 'panda_joint5', 'panda_joint6', 'panda_joint7', 'panda_joint8']

kindyn = KinDynComputations(urdfstring="panda.urdf", joints_name_list=joints_name_list)


Unknown tag "material" in /robot[@name='panda']/link[@name='panda_link0']/collision[1]
Unknown tag "material" in /robot[@name='panda']/link[@name='panda_link1']/collision[1]
Unknown tag "material" in /robot[@name='panda']/link[@name='panda_link2']/collision[1]
Unknown tag "material" in /robot[@name='panda']/link[@name='panda_link4']/collision[1]
Unknown tag "material" in /robot[@name='panda']/link[@name='panda_link5']/collision[1]
Unknown tag "material" in /robot[@name='panda']/link[@name='panda_link6']/collision[1]
Unknown tag "material" in /robot[@name='panda']/link[@name='panda_link7']/collision[1]
Unknown tag "material" in /robot[@name='panda']/link[@name='panda_hand']/collision[1]
Unknown tag "material" in /robot[@name='panda']/link[@name='panda_leftfinger']/collision[1]
Unknown tag "contact" in /robot[@name='panda']/link[@name='panda_leftfinger']
Unknown tag "material" in /robot[@name='panda']/link[@name='panda_rightfinger']/collision[1]
Unknown tag "contact" in /robot[@name='pan

In [14]:
class MujocoWrapper:
  # a simple wrapper to use mujoco as a simulator
  def __init__(self, model, joints_list=None):
    self.model = model
    self.data = mujoco.MjData(model)
    self.renderer = mujoco.Renderer(self.model)

  def set_qpos(self, qpos):
    self.data.qpos[:] = qpos
    mujoco.mj_forward(self.model, self.data)

  def get_qpos(self):
    return self.data.qpos[:]

  def render(self):
    mujoco.mj_forward(self.model, self.data)
    self.renderer.update_scene(self.data)
    return self.renderer.render()

  def step(self):
    mujoco.mj_step(self.model, self.data)

  def set_qvel(self, qvel):
    self.data.qvel[:] = qvel
    mujoco.mj_forward(self.model, self.data)


wrapper = MujocoWrapper(model)


In [15]:
# create an inverse kinematics MPC solver
# I know, an MPC solver is not the best way to do IK,
# but I want to show how to use the casadi interface

w_H_ee = kindyn.forward_kinematics_fun("panda_hand")

opti = cs.Opti()

N = 10 # number of control intervals
dt = 0.05 # time step

q = opti.variable(8, N+1)
q_dot = opti.variable(8, N)
w_H_b = np.eye(4)

des_ee_pos = opti.parameter(3)

q0 = opti.parameter(8)

opti.subject_to(q[:, 0] == q0)
target_cost = 0

for i in range(N):
    opti.subject_to(q[:, i+1] == q[:, i] + q_dot[:, i] * dt)
    opti.subject_to(opti.bounded(-5, q_dot[:, i], 5))
    target_cost += cs.sumsqr(w_H_ee(w_H_b, q[:, i])[:3, 3] - des_ee_pos) * dt

target_cost += cs.sumsqr(w_H_ee(w_H_b, q[:, -1])[:3, 3] - des_ee_pos)
velocities_cost = cs.sumsqr(q_dot) * 1e-3


opti.minimize(target_cost + velocities_cost)

p_opts = {"expand": True}
s_opts = {"max_iter": 100, "print_level": 0}
opti.solver("ipopt", p_opts, s_opts)


In [18]:
# start mujoco simulation along with control
duration = 10  # (seconds)
framerate = 60  # (Hz)

# Simulate and display video.

des_ee_pos_numeric = np.array([0.4, 1.0, 0.4])

# opti.set_value(des_ee_pos, des_ee_pos_numeric)

frames = []
mujoco.mj_resetData(wrapper.model, wrapper.data)
i = 0
while wrapper.data.time < duration:
  wrapper.step()
  if len(frames) < wrapper.data.time * framerate:
    # you do not need to set the desired ee position every time step
    # you can set it only when you want to change it
    opti.set_value(des_ee_pos, des_ee_pos_numeric)
    i += 1
    q0_numeric = wrapper.get_qpos()
    # remove the last joint since they are not controlled
    q0_numeric = q0_numeric[:-1]
    # set the initial condition
    opti.set_value(q0, q0_numeric)
    sol = opti.solve()
    # take the q_dot solution at the first time step and extend with the last joint to 0
    sol_q_dot = sol.value(q_dot)[:, 0]
    sol_q_dot = np.concatenate((sol_q_dot, np.zeros(1)))
    wrapper.set_qvel(sol_q_dot)
    pixels = wrapper.render()
    frames.append(pixels)
    # set the solution as the initial condition for the next time step
    opti.set_initial(q, sol.value(q))
    opti.set_initial(q_dot, sol.value(q_dot))
  if wrapper.data.time > 2:
    # change the desired ee position
    des_ee_pos_numeric = np.array([2.0, 0.0, 0.2])
    opti.set_value(des_ee_pos, des_ee_pos_numeric)
  if wrapper.data.time > 4:
    # change the desired ee position
    des_ee_pos_numeric = np.array([0.0, 0.5, 0.4])
    opti.set_value(des_ee_pos, des_ee_pos_numeric)
  if wrapper.data.time > 6:
    # change the desired ee position
    des_ee_pos_numeric = np.array([0.0, -0.6, 0.6])
    opti.set_value(des_ee_pos, des_ee_pos_numeric)

media.show_video(frames, fps=framerate)


      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  |  23.98ms ( 20.39us)  23.19ms ( 19.72us)      1176
       nlp_g  |  10.77ms (  9.16us)   8.26ms (  7.03us)      1176
    nlp_grad  |  11.43ms ( 62.11us)  11.40ms ( 61.95us)       184
  nlp_grad_f  |  62.21ms ( 45.78us)  61.98ms ( 45.60us)      1359
  nlp_hess_l  | 232.33ms (234.92us) 233.53ms (236.13us)       989
   nlp_jac_g  |   7.95ms (  5.85us)   7.89ms (  5.80us)      1359
       total  |  55.36ms ( 55.36ms)  67.60ms ( 67.60ms)         1
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  |  24.18ms ( 20.42us)  23.39ms ( 19.75us)      1184
       nlp_g  |  10.85ms (  9.17us)   8.33ms (  7.03us)      1184
    nlp_grad  |  11.50ms ( 62.17us)  11.47ms ( 62.01us)       185
  nlp_grad_f  |  62.69ms ( 45.83us)  62.45ms ( 45.65us)      1368
  nlp_hess_l  | 234.28ms (235.22us) 235.48ms (236.42us)       996
   nlp_jac_g  |   8.01ms (  5.86us)   7.95ms (  5.81us)      1368
       tot

This browser does not support the video tag.
